In [14]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib as mpl
import re
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [15]:
feature_df = pd.read_csv('./cleaned_csv/feature_df.csv',low_memory=False)
print('rows:',len(feature_df))

rows: 239850


In [16]:
feature_df.dropna(inplace = True)

In [17]:
feature_df.head(1)

,sentiment,cleaned_text,original_word_count,original_average_word_length,stopword_ratio
0,Neutral,big group make reservation week advance,0.12037,0.055914,0.533333


In [18]:
feature_df_bad = feature_df[feature_df.sentiment =="Poor"]
print('rows:',len(feature_df_bad))

rows: 79940


In [19]:
feature_df_good = feature_df[feature_df.sentiment =="Good"]
print('rows:',len(feature_df_good))

rows: 79948


## Topic Modelling

How to interpret this graph?
Topics on the left while their respective keywords are on the right.
Larger topics are more frequent and closer the topics, mor the similarity
Selection of keywords is based on their frequency and discriminancy.
Hover over the topics on the left to get information about their keywords on the right.

Reference: https://www.kaggle.com/thebrownviking20/topic-modelling-with-spacy-and-scikit-learn

In [20]:
#ngram for bad reviews

In [21]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [22]:
NUM_TOPICS = 10

In [23]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [24]:
bivectorizer = CountVectorizer(min_df=3, max_df=0.9, stop_words='english', lowercase=True, ngram_range=(2,3))
bigram_vectorized = bivectorizer.fit_transform(feature_df_bad["cleaned_text"])

In [25]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD


bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=20, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


In [26]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
[('long line', 516.4610457214435), ('bloody mary', 495.37917246405675), ('service terrible', 375.76532957285036), ('wait long', 263.2725901360335), ('food average', 187.78675924389984), ('spicy chicken', 177.36246770114428), ('prepare wait', 173.62386966146525), ('order food', 153.33269437373704), ('service rude', 149.17373808296813), ('wait food', 143.98352432173772)]
Topic 1:
[('wont disappoint', 448.74032076832634), ('park lot', 397.77537755070813), ('chicken salad', 291.00915963266897), ('service suck', 286.5033606424176), ('order chicken', 204.3091743927404), ('terrible food', 194.04745781466238), ('mac cheese', 186.36238035923873), ('secret menu', 174.8622765452566), ('dirty din', 151.9775924013386), ('ice cold', 131.9869479119521)]
Topic 2:
[('bad service', 763.3983120905232), ('service slow', 537.6612987767351), ('terrible service', 505.48345100332176), ('taste like', 493.8378911855421), ('food service', 342.3640262675626), ('chicken wing', 313.7603496199

In [27]:
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
pyLDAvis.enable_notebook()

bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
8      32.420952   8.115230       1        1  10.381030
4       0.648955  71.707100       2        1  10.123143
7      71.019684   9.971006       3        1  10.102106
2       3.785981  33.185135       4        1  10.032281
6     -42.666500  -9.222070       5        1  10.021027
3      -3.621689 -43.526516       6        1   9.975801
5      -3.632402  -4.171708       7        1   9.975208
0     -36.584400  35.804413       8        1   9.944024
9      44.391380  50.615383       9        1   9.753967
1      40.235809 -31.591665      10        1   9.691414, topic_info=                   Term         Freq        Total Category  logprob  loglift
7055   customer service  1272.000000  1272.000000  Default  30.0000  30.0000
27458      slow service   732.000000   732.000000  Default  29.0000  29.0000
1892        bad service   682.000000   682.000000  Default  28.0000  28.0000
13883  horrible service   629.000000   629.000000  Default  27.0000  27.0000
11915       fry chicken   537.000000   537.000000  Default  26.0000  26.0000
...                 ...          ...          ...      ...      ...      ...
14049         hot sauce    58.131125    58.933486  Topic10  -6.1379   2.3202
24899     salad chicken    55.814126    56.616472  Topic10  -6.1786   2.3197
11237         food make    54.326500    55.128896  Topic10  -6.2056   2.3193
11955          fry cold    54.036678    54.839085  Topic10  -6.2109   2.3192
5269      chicken tikka    53.307603    54.109909  Topic10  -6.2245   2.3190

[338 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
35         2  0.990254         10 min
45         2  0.993180      10 minute
175        1  0.991995         15 min
185        6  0.994972      15 minute
249        8  0.992444         20 min
...      ...       ...            ...
34008      5  0.997976  worst service
34109      3  0.994412    worth money
34116      3  0.994819    worth price
34137      7  0.989619     worth wait
34300      9  0.993762    wrong order

[310 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 8, 3, 7, 4, 6, 1, 10, 2])

In [ ]:
pyLDAvis.show(bi_dash)




Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [02/Aug/2020 16:14:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2020 16:14:14] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2020 16:14:14] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2020 16:14:14] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2020 16:14:14] code 404, message Not Found
127.0.0.1 - - [02/Aug/2020 16:14:14] "GET /favicon.ico HTTP/1.1" 404 -
